In [2]:
import warnings
warnings.filterwarnings('ignore')
import scrapy
from scrapy.crawler import CrawlerProcess
import re
import pandas as pd
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse


In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess

def parse_word(response):
    word = response.meta['word']
    
    # Extracting lexicons
    word['lexicon_0'] = response.xpath('//body/br[2]/following-sibling::text()').get()
    if word['lexicon_0'] and word['lexicon_0'].strip() == '':
        word['lexicon_0'] = response.xpath('//body/br[3]/following-sibling::text()').get()

    for i in range(1, 5):
        lex = response.xpath(f'//body/hr[{i}]/following-sibling::text()').get()
        if lex and lex.strip() != '':
            word[f'lexicon_{i}'] = lex.strip()
        else:
            word[f'lexicon_{i}'] = None

    # Extracting multiple word meanings
    meanings = response.xpath('//span[@class="mgP"]/text()').getall()
    for i, meaning in enumerate(meanings):
        # Limit to up to 4 meanings to avoid excessive columns
        if i < 4:
            word[f'meaning_{i}'] = meaning.strip()
    
    # Fill any remaining meaning columns with None
    for i in range(len(meanings), 4):
        word[f'meaning_{i}'] = None

    yield word

class CalWordSpider(scrapy.Spider):
    name = 'cal'
    # replace url
    start_urls = [
        'https://cal.huc.edu/get_a_chapter.php?file=71022'
    ]

    def parse(self, response):
        for el in response.css('tr > td:nth-child(2) > a'):
            word = {
                'text': el.css('::text').get(),
                'url': el.xpath('@href').get()
            }
            yield scrapy.Request(
                url=f"https://cal.huc.edu/{word['url']}",
                meta={'word': word},
                callback=parse_word
            )

# Run the spider
process = CrawlerProcess(settings={
    'FEEDS': {
        'Masekhet_bb.json': {
            'format': 'json',
            'overwrite': True
        }
    }
})

process.crawl(CalWordSpider)
process.start()
